In [ ]:
### REMOVE LATER ###
# Go up one directory since notebook inside task3/ folder.
import os

try:
    if UP_DIR:
        print("skipping")
except NameError:
    os.chdir("..")
    UP_DIR = True

# JH's Task 3 Experiment 1
(NOTE: keep this H1 header block or add it later to denote the boundaries between notebooks when we combined later)

I don't know what I am doing yet for this.

## Imports

In [2]:
from dataclasses import dataclass

import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA

import xgboost as xgb

## Hyperparameters

In [3]:
@dataclass
class Hparams:
    val_split: float = 0.2
    seed: int = 42

    pca_n_components = 1000


HP = Hparams()

In [ ]:
TRAIN_CSV = "./data/train.csv"
TRAIN_TFIDF_CSV = "./data/train_tfidf_features.csv"
TEST_CSV = "./data/test.csv"
TEST_TFIDF_CSV = "./data/test_tfidf_features.csv"

## Data Engineering
Instead of using the tfidf features given in the comp, we can engineer our own tfidf features with better filtering logic, or using something other than tfidf altogether to arrive at vector representations, or perhaps even use a strategy that use non-vector representations.

### Load Data

In [5]:
# train_df = pd.read_csv(TRAIN_CSV, index_col="id")
train_tfidf_df = pd.read_csv(TRAIN_TFIDF_CSV, index_col="id")
# test_df = pd.read_csv(TEST_CSV, index_col="id")
test_tfidf_df = pd.read_csv(TEST_TFIDF_CSV, index_col="id")

In [6]:
def tfidf_to_np(df: pd.DataFrame):
    """Convert the tfidf CSVs to X array of features and y array of labels, ordered
    by id.
    """
    df = df.sort_index()

    if "label" in df.columns:
        y = df.pop("label").to_numpy()
    else:
        y = None

    X = df.to_numpy()
    return X, y

In [7]:
train_X, val_X, train_y, val_y = train_test_split(
    *tfidf_to_np(train_tfidf_df), test_size=HP.val_split, random_state=HP.seed
)
test_X, test_y = tfidf_to_np(test_tfidf_df)

print("Dataset splits:")
print("  train:", len(train_X))
print("  val:  ", len(val_X))
print("  test: ", len(test_X))

Dataset splits:
  train: 13747
  val:   3437
  test:  4296


### Fit PCA For Dim Reduction

In [8]:
model_pca = PCA(n_components=HP.pca_n_components, random_state=HP.seed)

model_pca.fit(train_X)

,n_components,1000
,copy,True
,whiten,False
,svd_solver,'auto'
,tol,0.0
,iterated_power,'auto'
,n_oversamples,10
,power_iteration_normalizer,'auto'
,random_state,42


In [9]:
ratios = list(zip(range(HP.pca_n_components), model_pca.explained_variance_ratio_))
ratios.sort(key=lambda x: x[1], reverse=True)

print("Most Informative Dimensions:")
for idx, val in ratios[:3]:
    print(f"  Dim {idx}: {val}")

print("Noise:", model_pca.noise_variance_)
print("Total Explained Variance:", sum(model_pca.explained_variance_))

Most Informative Dimensions:
  Dim 0: 0.0101627373447088
  Dim 1: 0.006702354906393809
  Dim 2: 0.006077099322007794
Noise: 8.524432487454892e-05
Total Explained Variance: 0.6480273990858594


In [10]:
t_train_X = model_pca.transform(train_X)
t_val_X = model_pca.transform(val_X)
t_test_X = model_pca.transform(test_X)

# t_train_X = train_X.copy()
# t_val_X = val_X.copy()
# t_test_X = test_X.copy()

In [13]:
np.unique_counts(train_y)

UniqueCountsResult(values=array([0, 1]), counts=array([8480, 5267]))

## Fit XGBoost
Its boosted trees or random forests for both classification or regression.

Tutorial: <https://xgboost.readthedocs.io/en/stable/python/python_intro.html#setting-parameters>

In [12]:
dtrain = xgb.DMatrix(t_train_X, label=train_y)
dval = xgb.DMatrix(t_val_X, label=val_y)

In [14]:
# https://xgboost.readthedocs.io/en/stable/tutorials/param_tuning.html
params = {
    "validate_parameters": True,
    "colsample_bynode": 0.5,
    "learning_rate": 0.01,
    "max_depth": 6,
    "num_parallel_tree": 1, #  500,
    # TODO: when you use hinge loss cuz its easier to understand but actually logistic
    # regression converges faster, but actually you should just add another eval metric
    # "objective": "binary:hinge",
    "objective": "binary:logistic",
    "subsample": 0.5,
    "sampling_method": "gradient_based",
    "min_split_loss": 0.5,
    "min_child_weight": 2,
    "scale_pos_weight": 8480 / 5267,
    "lambda": 2,
    "tree_method": "hist",
    "device": "gpu",
    "eval_metric": "error",
}
num_round = 10000
early_stopping_rounds = 400
# Last set is used by xgb's early stopping.
eval_list = [(dtrain, "train"), (dval, "val")]

In [15]:
results = {}
bst = xgb.train(
    params,
    dtrain,
    num_round,
    evals=eval_list,
    evals_result=results,
    early_stopping_rounds=early_stopping_rounds,
)

[0]	train-error:0.34189	val-error:0.39395
[1]	train-error:0.33753	val-error:0.39831
[2]	train-error:0.31665	val-error:0.38289
[3]	train-error:0.30894	val-error:0.37358
[4]	train-error:0.30399	val-error:0.36834
[5]	train-error:0.29672	val-error:0.36427
[6]	train-error:0.29628	val-error:0.35758
[7]	train-error:0.28654	val-error:0.35583
[8]	train-error:0.28661	val-error:0.35991
[9]	train-error:0.28188	val-error:0.35409
[10]	train-error:0.28297	val-error:0.35176
[11]	train-error:0.28173	val-error:0.35147
[12]	train-error:0.27846	val-error:0.34623
[13]	train-error:0.27824	val-error:0.34332
[14]	train-error:0.27519	val-error:0.34420
[15]	train-error:0.27410	val-error:0.34769
[16]	train-error:0.27388	val-error:0.34565
[17]	train-error:0.27402	val-error:0.34798
[18]	train-error:0.27242	val-error:0.34798
[19]	train-error:0.27104	val-error:0.34158
[20]	train-error:0.27148	val-error:0.34158
[21]	train-error:0.26544	val-error:0.33663
[22]	train-error:0.26588	val-error:0.34129
[23]	train-error:0.26

In [15]:
# xgb.plot_importance(bst)

## Inference

In [16]:
dtest = xgb.DMatrix(t_test_X)

pred_y = bst.predict(dtest, iteration_range=(0, bst.best_iteration + 1))

In [ ]:
from datetime import datetime

pred_df = pd.DataFrame(
    zip(test_tfidf_df.index, np.where(pred_y > 0.5, 1, 0)), columns=["row ID", "label"]
)
# YYYYMMDD-HHMM.csv
pred_df.to_csv(f"{datetime.now().strftime('%Y%m%d-%H%M')}.csv", index=False)